# ODSC WEST 2020: Text Features In CatBoost
## Part 0 - CatBoost installation and notebook run

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/events/2020_odsc_west/text_features.ipynb)

**Set GPU as hardware accelerator**

First of all, you need to select GPU as hardware accelerator. There are two simple steps to do so:
Step 1. Navigate to **Runtime** menu and select **Change runtime type**
Step 2. Choose **GPU** as hardware accelerator.
That's all!

Let's install CatBoost.

In [ ]:
# Install essentials:
!pip install -U catboost matplotlib sklearn shap

# Install widgets
!pip install -U ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import catboost
print(catboost.__version__)

# Part 1: CatBoost base tutorial

In [ ]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

## Reading the data

In [ ]:
from catboost.datasets import msrank_10k

# If you have "URLError: SSL: CERTIFICATE_VERIFY_FAILED" uncomment next two lines:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

(train_df, test_df) = msrank_10k()

In [ ]:
train_df.head()

## Preparing the data

Label values extraction

In [ ]:
y = train_df[0]
X = train_df.drop([0, 1], axis=1)

Ways to create Pool class. If you have a big dataset it is effective (in terms of time) to load data from file, instead of pandas Dataframe.

In [ ]:
dataset_dir = './msrank_10k'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

train_df.to_csv(
    os.path.join(dataset_dir, 'train.csv'),
    index=False, sep=',', header=True
)
test_df.to_csv(
    os.path.join(dataset_dir, 'test.csv'),
    index=False, sep=',', header=True
)

In [ ]:
!head -2 msrank_10k/train.csv

In [ ]:
from catboost.utils import create_cd
feature_names = dict(map(lambda i: (i + 2, 'Feature ' + str(i)), range(train_df.shape[1])))
    
create_cd(
    label=0,
    feature_names=feature_names,
    auxiliary_columns=[1],
    output_path=os.path.join(dataset_dir, 'train.cd')
)

In [ ]:
from catboost import Pool

pool1 = Pool(data=X, label=y)

pool2 = Pool(
    data=os.path.join(dataset_dir, 'train.csv'), 
    delimiter=',', 
    column_description=os.path.join(dataset_dir, 'train.cd'),
    has_header=True,
)

print('Dataset shape: {}\n'.format(pool1.shape))

## Split your data into train and validation

In [ ]:
from sklearn.model_selection import train_test_split

data = train_test_split(X, y, train_size=0.8, random_state=0)
X_train, X_validation, y_train, y_validation = data

train_pool = Pool(data=X_train, label=y_train)
validation_pool = Pool(data=X_validation, label=y_validation)

## Training

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    iterations=5,
    learning_rate=0.1,
)
model.fit(train_pool, eval_set=validation_pool, verbose=False)

print('Model is fitted: {}'.format(model.is_fitted()))
print('Model params:\n{}'.format(model.get_params()))

## Stdout of the training

In [ ]:
model = CatBoostRegressor(
    iterations=15,
#     verbose=5,
)
model.fit(train_pool, eval_set=validation_pool);

## Metrics calculation and graph plotting

In [ ]:
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.2,
    custom_metric=['MAE', 'R2']
)

model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False,
    plot=True
);

## Best iteration

In [ ]:
model = CatBoostRegressor(
    iterations=100,
    eval_metric='MAE',
    learning_rate=0.5,
#     use_best_model=False
)
model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False,
    plot=True
);

In [ ]:
print('Tree count: ' + str(model.tree_count_))

## Grid Search

In [ ]:
pool = Pool(data=X_train, label=y_train)
model = CatBoostRegressor(iterations=10, eval_metric='MAE')
grid = {'learning_rate': [0.001, 0.01, 0.1], 'depth': [4, 5, 6]}
result = model.grid_search(grid, pool)

In [ ]:
result

In [ ]:
print('Best parameters: {}\n'.format(result['params']))

msg = 'Mean MAE value on validation set per each iteration:\n{}'
print(msg.format(np.round(result['cv_results']['test-MAE-mean'], 4)))

In [ ]:
model.get_params()

In [ ]:
model.predict(validation_pool)

In [ ]:
model = CatBoostRegressor(iterations=100, eval_metric='MAE')
model.grid_search(grid, pool, plot=True, verbose=False);

More about parameter tuning you can find in [tutorial](https://github.com/catboost/catboost/blob/master/catboost/tutorials/hyperparameters_tuning/hyperparameters_tuning.ipynb).

## Feature importances

### Prediction values change

In [ ]:
model.get_feature_importance()

In [ ]:
model.get_feature_importance(prettified=True)

### Loss function change

In [ ]:
model.get_feature_importance(
    data=validation_pool, 
    type='LossFunctionChange',
    prettified=True
).head()

### Shap values

In [ ]:
model = CatBoostRegressor(iterations=1000, learning_rate=0.1)
model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False,
    plot=True
);


shap_values = model.get_feature_importance(
    data=train_pool, 
    type='ShapValues'
)

expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]

print(shap_values.shape)

In [ ]:
import shap

shap.initjs()
shap.force_plot(
    expected_value,
    shap_values[1,:],
    feature_names=train_pool.get_feature_names()
)

In [ ]:
shap.force_plot(
    expected_value,
    shap_values[7,:],
    feature_names=train_pool.get_feature_names()
)

In [ ]:
shap.summary_plot(shap_values, X_train)

More information about shap value usage you can find in [tutorial](https://github.com/catboost/catboost/blob/master/catboost/tutorials/model_analysis/shap_values_tutorial.ipynb).

## Tree Visualization

In [ ]:
from sklearn.datasets import make_regression

model = CatBoostRegressor(
    iterations=2,
    depth=2,
    learning_rate=0.5,
    boost_from_average=False)

features, labels = make_regression(n_samples=50, n_features=4, n_informative=2)

model.fit(features, labels, verbose=False);

In [ ]:
# This cell doesn't work without graphviz package
# You can install it by link https://graphviz.gitlab.io/download/
# Installation can take a lot of time. You can do it at home.

from IPython.display import display

display(model.plot_tree(0))
display(model.plot_tree(1))

In [ ]:
x = [0, 0,0,0]

raw_pred = model.predict([x])
print(raw_pred)

## Grow Policies: SymmetricTree, Depthwise, Lossguide

In [ ]:
model = CatBoostRegressor(iterations=2, depth=3, grow_policy='SymmetricTree')
model.fit(features, labels, verbose=False);
display(model.plot_tree(0))

In [ ]:
model = CatBoostRegressor(iterations=2, depth=3, grow_policy='Depthwise')
model.fit(features, labels, verbose=False);
display(model.plot_tree(0))

In [ ]:
model = CatBoostRegressor(iterations=2, max_leaves=8, grow_policy='Lossguide')
model.fit(features, labels, verbose=False);
display(model.plot_tree(0))

## Data uncertainty

In [ ]:
X = np.arange(-10,10, 0.0005, dtype=np.float32)
Y = np.arctan(X)
for i in range(len(X)):
    Y[i] = np.random.normal(Y[i], np.sin(X[i]/1.5) ** 2 /2)

model = CatBoostRegressor(iterations=1000, learning_rate=0.1, loss_function='RMSEWithUncertainty',
                          verbose=100, random_seed=0, border_count=1024, posterior_sampling=True)
Xs = X.reshape((len(X), 1))

model.fit(Xs, Y)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
vapred = model.virtual_ensembles_predict(Xs, prediction_type='TotalUncertainty', virtual_ensembles_count=5)

plt.figure(figsize = (16,9))

plt.plot(X, Y, '.', c='gray', markersize=2)
plt.plot(X, vapred[:, 0],c='magenta')
plt.plot(X, vapred[:, 0] + np.sqrt(vapred[:, 1] + vapred[:, 2]), c='blue')
plt.plot(X, vapred[:, 0] - np.sqrt(vapred[:, 1] + vapred[:, 2]), c='blue')

## Snapshotting

In [ ]:
!rm 'catboost_info/snapshot.bkp'

In [ ]:
model = CatBoostRegressor(
    iterations=3000,
    save_snapshot=True,
    snapshot_file='snapshot.bkp',
    snapshot_interval=1
)

model.fit(train_pool, eval_set=validation_pool, verbose=100);

## Saving the model

In [ ]:
model = CatBoostRegressor(iterations=10)
model.fit(train_pool, eval_set=validation_pool, verbose=False)
model.save_model('catboost_model.bin')
model.save_model('catboost_model.json', format='json')

In [ ]:
model.load_model('catboost_model.bin')
print(model.get_params())
print(model.learning_rate_)

# **Questions and answers?**

# Part 2: CatBoost Text features support

In this tutorial we will use dataset **Rotten Tomatoes Movie Reviews** from [Kaggle](https://www.kaggle.com) competition for our experiments. Data can be downloaded [here](https://www.kaggle.com/rpnuser8182/rotten-tomatoes/data).

In [ ]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

## Reading the data

In [ ]:
from catboost.datasets import rotten_tomatoes

train_df, test_df = rotten_tomatoes()

train_df.head(2)

### Features description 

|Id | Feature name      |  Description                                                                                 |
|---|-------------------|----------------------------------------------------------------------------------------------|
| 1 | ``id``            |  unique movie id                                                                             |
| 2 | ``synopsis``      |  brief summary of the major points of a movie                                                |
| 3 | ``rating_MPAA``   |  film rating by MPAA rating system                                                           |
| 4 | ``genre``         |  list of genres that are suitable for this film (e.g. Action, Adventure, Comedy,...          |
| 5 | ``director``      |  list of persons who direct the making of a film                                             |
| 6 | ``writer``        |  list of persons who write a screenplay                                                      |
| 7 | ``theater_date``  |  the date when film was first shown to the public in cinema (string)                         |
| 8 | ``dvd_date``      |  the date when film was released on DVD (string)                                             |
| 9 | ``box_office``    |  the amount of money raised by ticket sales (revenue)                                        |
| 10 | ``runtime``      |  film duration in minutes                                                                    |
| 11 | ``studio``       |  is a major entertainment company or motion picture company (20th Century Fox, Sony Pictures)|
| 12 | ``dvd_date_int`` |  the date when film was released on DVD (converted to integer)                               |
| 13 | ``theater_date_int`` |  the date when film was first shown to the public in cinema (converted to integer)       |
| 14 | ``review``       |  review of a movie, that was written by a critic                                             |
| 15 | ``rating``       |  float rating from 0 to 1 of the film according to the Rotten tomatoes web site              |
| 16 | ``fresh``        |  freshness of review - fresh or rotten                                                       |
| 17 | ``critic``       |  name of reviewer                                                                            |
| 18 | ``top_critic``   |  binary feature, is reviewer a top critic or not                                             |
| 19 | ``publisher``    |  journal or website where the review was published                                           |
| 20 | ``date``         |  the date when critic publish review (string)                                                |
| 21 | ``date_int``     |  the date when critic publish review (converted to integer)                                  |
| 22 | ``rating_10``    |  integer rating from 0 to 10 of the film according to the critic                             |

We mark as **auxiliary** columnns 'id' and 'rating', because they can be the reason of overfitting, 'theater_date','dvd_date','date' because we convert them into integers.

We mark as **text** features 'synopsis' because it is short *text* description of a film, 'genre' because it is combination of categories (we know that strings have structure where words define categories), for example 'Action | Comedy | Adventure', 'director' and 'writer' features are included to the text features by the same reason, 'review' becuase it is a *text* summary of critic opinion.

We mark as **categorical** features 'rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic' and 'publisher' because they can not be splitted into the group of categorical features and feature values can not be compared.

The other columns considered as **numeric**.

## Preparing the data

In [ ]:
auxiliary_columns = ['id', 'theater_date', 'dvd_date', 'rating', 'date']
cat_features = ['rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic', 'publisher']
text_features = ['synopsis', 'genre', 'director', 'writer', 'review']

In [ ]:
def fill_na(df, features):
    for feature in features:
        df[feature].fillna('', inplace=True)

def preprocess_data_part(data_part):
    data_part = data_part.drop(auxiliary_columns, axis=1)

    fill_na(data_part, cat_features)
    fill_na(data_part, text_features)

    X = data_part.drop(['rating_10'], axis=1)
    y = data_part['rating_10']
    return X, y

X_train, y_train = preprocess_data_part(train_df)
X_test, y_test = preprocess_data_part(test_df)

X_train_no_text = X_train.drop(text_features, axis=1)
X_test_no_text = X_test.drop(text_features, axis=1)

In [ ]:
X_train_no_text.head(2)

In [ ]:
from catboost import Pool

train_pool_no_text = Pool(
    X_train_no_text, y_train, 
    cat_features=cat_features, 
)

validation_pool_no_text = Pool(
    X_test_no_text, y_test, 
    cat_features=cat_features, 
)

print('Train dataset shape: {}\n'.format(train_pool_no_text.shape))

In [ ]:
from catboost import CatBoostClassifier

def fit_model(train_pool, validation_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        eval_metric='Accuracy',
        task_type='GPU',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=validation_pool,
        verbose=100,
    )

model_no_text = fit_model(train_pool_no_text, validation_pool_no_text)

# Text Features

In [ ]:
train_pool = Pool(
    X_train, y_train, 
    cat_features=cat_features,
    text_features=text_features,
)

validation_pool = Pool(
    X_test, y_test, 
    cat_features=cat_features,
    text_features=text_features,
)

print('Train dataset shape: {}\n'.format(train_pool.shape))

model = fit_model(train_pool, validation_pool)

In [ ]:
def print_score_diff(first_model, second_model):
    first_accuracy = first_model.best_score_['validation']['Accuracy']
    second_accuracy = second_model.best_score_['validation']['Accuracy']

    gap = (second_accuracy - first_accuracy) / first_accuracy * 100

    print('{} vs {} ({:+.2f}%)'.format(first_accuracy, second_accuracy, gap))

In [ ]:
print_score_diff(model_no_text, model)

<span style="color:red">Note!</span>

1. Text features also cannot contain NaN values, so we converted them into strings manually.
2. The training may be performed only with classification losses and targets.

## How it works?

1. **Text Tokenization**
2. **Dictionary Creation**
3. **Feature Calculation**

## Text Tokenization

Usually we get our text as a sequence of Unicode symbols. So, if the task isn't a DNA classification we don't need such granularity, moreover, we need to extract more complicated entities, e.g. words. The process of extraction tokens -- words, numbers, punctuation symbols or special symbols which defines emoji from a sequence is called **tokenization**.<br>

Tokenization is the first part of text preprocessing in CatBoost and performed as a simple splitting a sequence on a string pattern (e.g. space).

In [ ]:
text_small = [
    "Cats are so cute :)",
    "Mouse scare...",
    "The cat defeated the mouse",
    "Cute: Mice gather an army!",
    "Army of mice defeated the cat :(",
    "Cat offers peace",
    "Cat is scared :(",
    "Cat and mouse live in peace :)"
]

target_small = [1, 0, 1, 1, 0, 1, 0, 1]

In [ ]:
from catboost.text_processing import Tokenizer

simple_tokenizer = Tokenizer()

def tokenize_texts(texts):
    return [simple_tokenizer.tokenize(text) for text in texts]

simple_tokenized_text = tokenize_texts(text_small)
simple_tokenized_text

### More preprocessing!

Lets take a closer look on the tokenization result of small text example -- the tokens contains a lot of mistakes:

1. They are glued with punctuation 'Cute:', 'army!', 'skare...'.
2. The words 'Cat' and 'cat', 'Mice' and 'mice' seems to have same meaning, perhaps they should be the same tokens.
3. The same problem with tokens 'are'/'is' -- they are inflected forms of same token 'be'.

**Punctuation handling**, **lowercasing**, and **lemmatization** processes help to overcome these problems.

### Punctuation handling and lowercasing

In [ ]:
tokenizer = Tokenizer(
    lowercasing=True,
    separator_type='BySense',
    token_types=['Word', 'Number']
)

tokenized_text = [tokenizer.tokenize(text) for text in text_small]
tokenized_text

### Removing stop words

**Stop words** - the words that are considered to be uninformative in this task, e.g. function words such as *the, is, at, which, on*.
Usually stop words are removed during text preprocessing to reduce the amount of information that is considered for further algorithms.
Stop words are collected manually (in dictionary form) or automatically, for example taking the most frequent words.

In [ ]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

### Lemmatization

Lemma (Wikipedia) -- is the canonical form, dictionary form, or citation form of a set of words.<br>
For example, the lemma "go" represents the inflected forms "go", "goes", "going", "went", and "gone".<br>
The process of convertation word to its lemma called **lemmatization**.


In [ ]:
import nltk

nltk_data_path = os.path.join(os.path.dirname(nltk.__file__), 'nltk_data')
nltk.data.path.append(nltk_data_path)
nltk.download('wordnet', nltk_data_path)

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_tokens_nltk(tokens):
    return list(map(lambda t: lemmatizer.lemmatize(t), tokens))

In [ ]:
text_small_lemmatized_nltk = [lemmatize_tokens_nltk(tokens) for tokens in tokenized_text_no_stop]
text_small_lemmatized_nltk

Now words with same meaning represented by the same token, tokens are not glued with punctuation.

<span style="color:red">Be carefull.</span> You should verify for your own task:<br>
Is it realy necessary to remove punctuation, lowercasing sentences or performing a lemmatization and/or by word tokenization?<br>

### Let's check up accuracy with new text preprocessing

Since CatBoost doesn't perform spacing punctuation, lowercasing letters and lemmatization, we need to preprocess text manually and then pass it to learning algorithm.

Since the natural text features is only synopsis and review, we will preprocess only them.

In [ ]:
%%time

def preprocess_data(X):
    X_preprocessed = X.copy()
    for feature in ['synopsis', 'review']:
        X_preprocessed[feature] = X[feature].apply(lambda x: ' '.join(lemmatize_tokens_nltk(tokenizer.tokenize(x))))
    return X_preprocessed

X_preprocessed_train = preprocess_data(X_train)
X_preprocessed_test = preprocess_data(X_test)

train_processed_pool = Pool(
    X_preprocessed_train, y_train, 
    cat_features=cat_features,
    text_features=text_features,
)

validation_processed_pool = Pool(
    X_preprocessed_test, y_test, 
    cat_features=cat_features,
    text_features=text_features,
)

In [ ]:
model_on_processed_data = fit_model(train_processed_pool, validation_processed_pool)

In [ ]:
print_score_diff(model, model_on_processed_data)

## Dictionary Creation

After the first stage, preprocessing of text and tokenization, the second stage starts. The second stage uses the prepared text to select a set of units, which will be used for building new numerical features.

A set of selected units is called dictionary. It might contain words, word bigramms, or character n-gramms.

In [ ]:
from catboost.text_processing import Dictionary

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=10)

dictionary.fit(text_small_lemmatized_nltk);
#dictionary.fit(text_small, tokenizer)

In [ ]:
dictionary.save('dictionary.tsv')
!cat dictionary.tsv

## Feature Calculation

### Convertation into fixed size vectors

The majority of classic ML algorithms are computing and performing predictions on a fixed number of features $F$.<br>
That means that learning set $X = \{x_i\}$ contains vectors $x_i = (a_0, a_1, ..., a_F)$ where $F$ is constant.

Since text object $x$ is not a fixed length vector, we need to perform preprocessing of the origin set $D$.<br>
One of the simplest text to vector encoding technique is **Bag of words (BoW)**.

### Bag of words algorithm

The algorithm takes in a dictionary and a text.<br>
During the algorithm text $x = (a_0, a_1, ..., a_k)$ converted into vector $\tilde x = (b_0, b_1, ..., b_F)$,<br> where $b_i$ is 0/1 (depending on whether there is a word with id=$i$ from dictionary into text $x$).

In [ ]:
text_small_lemmatized_nltk

In [ ]:
dictionary.apply([text_small_lemmatized_nltk[0]])

In [ ]:
def bag_of_words(tokenized_text, dictionary):
    features = np.zeros((len(tokenized_text), dictionary.size))
    for i, tokenized_sentence in enumerate(tokenized_text):
        indices = np.array(dictionary.apply([tokenized_sentence])[0])
        features[i, indices] = 1
    return features

bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
bow_features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
from sklearn.metrics import log_loss

def fit_linear_model(X, c):
    model = LogisticRegression()
    model.fit(X, c)
    return model

def fit_naive_bayes(X, c):
    clf = MultinomialNB()
    if isinstance(X, csr_matrix):
        X.eliminate_zeros()
    clf.fit(X, c)
    return clf

def evaluate_model_logloss(model, X, y):
    y_pred = model.predict_proba(X)[:,1]
    metric = log_loss(y, y_pred)
    print('Logloss: ' + str(metric))

In [ ]:
def evaluate_models(X, y):
    linear_model = fit_linear_model(bow_features, target_small)
    naive_bayes = fit_naive_bayes(bow_features, target_small)
        
    print('Linear model')
    evaluate_model_logloss(linear_model, X, y)
    print('Naive bayes')
    evaluate_model_logloss(naive_bayes, X, y)
    print('Comparing to constant prediction')
    logloss_constant_prediction = log_loss(y, np.ones(shape=(len(text_small), 2)) * 0.5)
    print('Logloss: ' + str(logloss_constant_prediction))
    
evaluate_models(bow_features, target_small)

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0)
dictionary.fit(text_small_lemmatized_nltk)

bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
evaluate_models(bow_features, target_small)

### Looking at sequences of letters / words

Let's look at the example: texts 'The cat defeated the mouse' and 'Army of mice defeated the cat :('<br>
Simplifying it we have three tokens in each sentence 'cat defeat mouse' and 'mouse defeat cat'.<br>
After applying BoW we get two equal vectors with the opposite meaning:

| cat | mouse | defeat |
|-----|-------|--------|
| 1   | 1     | 1      |
| 1   | 1     | 1      |

How to distinguish them?
Lets add sequences of words as a single tokens into our dictionary:

| cat | mouse | defeat | cat_defeat | mouse_defeat | defeat_cat | defeat_mouse |
|-----|-------|--------|------------|--------------|------------|--------------|
| 1   | 1     | 1      | 1          | 0            | 0          | 1            |
| 1   | 1     | 1      | 0          | 1            | 1          | 0            |

**N-gram** is a continguous sequence of $n$ items from a given sample of text or speech (Wikipedia).<br>
In example above Bi-gram (Bigram) = 2-gram of words.

Ngrams help to add into vectors more information about text structure, moreover there are n-grams has no meanings in separation, for example, 'Mickey Mouse company'.

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary.fit(text_small_lemmatized_nltk)

dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [ ]:
bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
evaluate_models(bow_features, target_small)

### Unigram + Bigram

In [ ]:
dictionary1 = Dictionary(occurence_lower_bound=0)
dictionary1.fit(text_small_lemmatized_nltk)

bow_features1 = bag_of_words(text_small_lemmatized_nltk, dictionary1)

dictionary2 = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary2.fit(text_small_lemmatized_nltk)

bow_features2 = bag_of_words(text_small_lemmatized_nltk, dictionary2)

bow_features = np.concatenate((bow_features1, bow_features2), axis=1)
evaluate_models(bow_features, target_small)

## CatBoost Configuration

Parameter names:

1. **Text Tokenization** - `tokenizers`
2. **Dictionary Creation** - `dictionaries`
3. **Feature Calculation** - `feature_calcers`

\* More complex configuration with `text_processing` parameter

### `tokenizers`

Tokenizers used to preprocess Text type feature columns before creating the dictionary.

[Documentation](https://catboost.ai/docs/references/tokenizer_options.html).

```
tokenizers = [{
	'tokenizer_id': 'Space',
	'delimiter': ' ',
	'separator_type': 'ByDelimiter',
},{
	'tokenizer_id': 'Sense',
	'separator_type': 'BySense',
}]
```

### `dictionaries`

Dictionaries used to preprocess Text type feature columns.

[Documentation](https://catboost.ai/docs/references/dictionaries_options.html).

```
dictionaries = [{
	'dictionary_id': 'Unigram',
	'max_dictionary_size': '50000',
	'gram_count': '1',
},{
	'dictionary_id': 'Bigram',
	'max_dictionary_size': '50000',
	'gram_count': '2',
},{
	'dictionary_id': 'Trigram',
	'token_level_type': 'Letter',
	'max_dictionary_size': '50000',
	'gram_count': '3',
}]
```

### `feature_calcers`

Feature calcers used to calculate new features based on preprocessed Text type feature columns.

1. **`BoW`**<br>
Bag of words: 0/1 features (text sample has or not token_id).<br>
Number of produced numeric features = dictionary size.<br>
Parameters: `top_tokens_count` - maximum number of tokens that will be used for vectorization in bag of words, the most frequent $n$ tokens are taken (**highly affect both on CPU ang GPU RAM usage**).

2. **`NaiveBayes`**<br>
NaiveBayes: [Multinomial naive bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) model. As many new features as classes are added. This feature is calculated by analogy with counters in CatBoost by permutation ([estimation of CTRs](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html)). In other words, a random permutation is made and then we go from top to bottom on the dataset and calculate the probability of its belonging to this class for each object.

3. **`BM25`**<br>
[BM25](https://en.wikipedia.org/wiki/Okapi_BM25). As many new features as classes are added. The idea is the same as in Naive Bayes, but for each class we calculate not the conditional probability, but a certain relevance, which is similar to tf-idf, where the tokens instead of the words and the classes instead of the documents (or rather, the unification of all texts of this class). Only the tf multiplier in BM25 is replaced with another multiplier, which gives an advantage to classes that contain rare tokens.

```
feature_calcers = [
	'BoW:top_tokens_count=1000',
	'NaiveBayes',
	'BM25',
]
```

### `text_processing`

```
text_processing = {
    "tokenizers" : [{
        "tokenizer_id" : "Space",
        "separator_type" : "ByDelimiter",
        "delimiter" : " "
    }],

    "dictionaries" : [{
        "dictionary_id" : "BiGram",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "2"
    }, {
        "dictionary_id" : "Word",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "1"
    }],

    "feature_processing" : {
        "default" : [{
            "dictionaries_names" : ["BiGram", "Word"],
            "feature_calcers" : ["BoW"],
            "tokenizers_names" : ["Space"]
        }, {
            "dictionaries_names" : ["Word"],
            "feature_calcers" : ["NaiveBayes"],
            "tokenizers_names" : ["Space"]
        }],
    }
}
```

In [ ]:
model_on_processed_data_2 = fit_model(
    train_processed_pool,
    validation_processed_pool,
    text_processing = {
        "tokenizers" : [{
            "tokenizer_id" : "Space",
            "separator_type" : "ByDelimiter",
            "delimiter" : " "
        }],
    
        "dictionaries" : [{
            "dictionary_id" : "BiGram",
            "max_dictionary_size" : "50000",
            "occurrence_lower_bound" : "3",
            "gram_order" : "2"
        }, {
            "dictionary_id" : "Word",
            "max_dictionary_size" : "50000",
            "occurrence_lower_bound" : "3",
            "gram_order" : "1"
        }],
    
        "feature_processing" : {
            "default" : [{
                "dictionaries_names" : ["BiGram", "Word"],
                "feature_calcers" : ["BoW"],
                "tokenizers_names" : ["Space"]
            }, {
                "dictionaries_names" : ["Word"],
                "feature_calcers" : ["NaiveBayes"],
                "tokenizers_names" : ["Space"]
            }],
        }
    }
)

In [ ]:
print_score_diff(model_no_text, model_on_processed_data_2)

# Summary: Text features in CatBoost

### The algorithm:
1. Input text is loaded as a usual column. ``text_column: [string]``.
2. Each text sample is tokenized via splitting by space. ``tokenized_column: [[string]]``.
3. Dictionary estimation.
4. Each string in tokenized column is converted into token_id from dictionary. ``text: [[token_id]]``.
5. Depending on the parameters CatBoost produce features basing on the resulting text column: Bag of words, Multinomial naive bayes or Bm25.
6. Computed float features are passed into the usual CatBoost learning algorithm.